In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict

In [2]:
df = pd.read_csv(R'D:\calorie_RL\data\meal3_cleaned.csv')
df

,meal_name,ingredients,calories_per_serving,carbs,proteins,fats,diet_type,meal_time,serving_size,cost_per_serving_in_inr,region
0,Chole Bhature,"Chickpeas, Refined Flour, Spices, Onion",450,55.0,15.0,18.0,Vegetarian,NaN,1 plate,50.0,North Indian
1,Aloo Paratha,"Wheat Flour, Potato, Spices, Butter",290,35.0,6.0,12.0,Vegetarian,NaN,1 paratha,25.0,North Indian
2,Butter Chicken,"Chicken, Butter, Cream, Tomato, Spices",490,18.0,28.0,32.0,Non-Vegetarian,dinner,1 bowl,80.0,North Indian
3,Kadhi Pakora,"Besan, Curd, Spices, Onion",260,20.0,8.0,16.0,Vegetarian,NaN,1 bowl,30.0,North Indian
4,Litti Chokha,"Wheat Flour, Sattu, Brinjal, Potato, Tomato",360,50.0,10.0,12.0,Vegetarian,NaN,3 littis with chokha,25.0,North Indian
...,...,...,...,...,...,...,...,...,...,...,...
483,Oats Porridge,"Oats, Milk, Sugar/Honey",150,25.0,5.0,3.0,Vegetarian,Breakfast,1 bowl,30.0,North Indian
484,Sweet Potato Chaat,"Sweet Potato, Spices, Lemon Juice",180,30.0,3.0,5.0,Vegetarian,Snack,1 bowl,30.0,North Indian
485,"Fruit Salad (Banana, apple, mango,blueberry)","Banana, apple, mango,blueberry",120,30.0,2.0,1.0,vegetarian,Snack,1 bowl,60.0,North Indian
486,Basmati Rice,"Basmati Rice, Water",220,45.0,5.0,2.0,Vegetarian,Lunch,1 cup cooked,30.0,North Indian


In [3]:
# Filter meals by diet_type
def filter_meals_by_diet(diet_type):
    return df[df['diet_type'] == diet_type]


In [33]:
class DietEnvironment:
    def __init__(self, meals, total_calories, total_budget):
        self.meals = meals
        self.total_calories = total_calories
        self.total_budget = total_budget
        self.current_calories = 0
        self.current_budget = 0
        self.selected_meals = []

    def reset(self):
        self.current_calories = 0
        self.current_budget = 0
        self.selected_meals = []
        return self.get_state()

    def get_state(self):
        return (self.current_calories, self.current_budget)

    def step(self, action):
        meal = self.meals.iloc[action]
        calories = meal['calories_per_serving']
        cost = meal['cost_per_serving_in_inr']

        # Add meal to the plan
        self.selected_meals.append(meal['meal_name'])
        self.current_calories += calories
        self.current_budget += cost

        # Calculate reward
        if self.current_calories > self.total_calories + 100 or self.current_budget > self.total_budget:
            reward = -50  # Penalty for exceeding constraints
            done = True
        elif self.total_calories - 100 <= self.current_calories <= self.total_calories + 100:
            reward = 50  # High reward for meeting calorie goal
            done = True
        else:
            reward = -5  # Small penalty for not meeting the goal yet
            done = False

        return self.get_state(), reward, done

In [34]:
class DietAgent:
    def __init__(self, n_actions):
        self.n_actions = n_actions
        self.q_table = defaultdict(lambda: np.zeros(self.n_actions))
        self.epsilon = 1.0  # Start with high exploration
        self.epsilon_min = 0.1  # Minimum exploration rate
        self.epsilon_decay = 0.995  # Decay rate
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor

    def choose_action(self, state):
        state = str(state)
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, self.n_actions - 1)  # Explore
        else:
            return np.argmax(self.q_table[state])  # Exploit

    def learn(self, state, action, reward, next_state):
        state = str(state)
        next_state = str(next_state)
        predict = self.q_table[state][action]
        target = reward + self.gamma * np.max(self.q_table[next_state])
        self.q_table[state][action] += self.alpha * (target - predict)

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
def train_agent(diet_type, total_calories, total_budget, episodes=5000):  # Increase episodes
    meals = filter_meals_by_diet(diet_type)
    if meals.empty:
        raise ValueError(f"No meals found for diet type: {diet_type}")

    env = DietEnvironment(meals, total_calories, total_budget)
    agent = DietAgent(len(meals))

    for _ in range(episodes):
        state = env.reset()
        done = False

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)
            agent.learn(state, action, reward, next_state)
            state = next_state

    return agent

In [29]:
def generate_diet_plan(agent, diet_type, total_calories, total_budget):
    meals = filter_meals_by_diet(diet_type)
    env = DietEnvironment(meals, total_calories, total_budget)
    state = env.reset()
    done = False

    while not done:
        action = agent.choose_action(state)
        state, _, done = env.step(action)

    # Ensure the calorie constraint is satisfied
    if not (total_calories - 100 <= env.current_calories <= total_calories + 100):
        raise ValueError("Generated diet plan does not satisfy the calorie constraint.")

    # Return detailed information about the selected meals
    selected_meals_details = meals[meals['meal_name'].isin(env.selected_meals)]
    return selected_meals_details

In [32]:
diet_type = 'Vegetarian'
total_calories = 2000
total_budget = 500

agent = train_agent(diet_type, total_calories, total_budget)
diet_plan_details = generate_diet_plan(agent, diet_type, total_calories, total_budget)

pd.DataFrame(diet_plan_details)

Initializing state in q_table: (0, 0)
Initializing state in q_table: (np.int64(740), np.float64(150.0))
Initializing state in q_table: (np.int64(1190), np.float64(200.0))
Initializing state in q_table: (np.int64(1640), np.float64(250.0))


,meal_name,ingredients,calories_per_serving,carbs,proteins,fats,diet_type,meal_time,serving_size,cost_per_serving_in_inr,region
0,Chole Bhature,"Chickpeas, Refined Flour, Spices, Onion",450,55.0,15.0,18.0,Vegetarian,NaN,1 plate,50.0,North Indian
18,Vegetable Biryani,"Basmati Rice, Mixed Vegetables, Yogurt, Ginger...",420,9.0,15.0,55.0,Vegetarian,NaN,1 plate,100.0,North Indian
185,Sohan halwa,"Corn flour, ghee, dry fruits",320,40.0,6.0,16.0,Vegetarian,Dessert,1 piece,50.0,Other
